# Neighborhood Computations

In [2]:
import numpy as np
import igl
import meshplot

In [3]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
bumpy_v, bumpy_f = igl.read_triangle_mesh("data/bumpy.off")
car_v, car_f = igl.read_triangle_mesh("data/car.off")
coffeecup_v, coffeecup_f = igl.read_triangle_mesh("data/coffeecup.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

In [ ]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

In [ ]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

In [ ]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

## Vertex-to-Face Relations

In [4]:
def vertex_to_face(f: np.array, v_n: int) -> dict:
    VF: np.array
    NI: np.array
    VF, NI = igl.vertex_triangle_adjacency(f, v_n)

    vertex_to_face = dict()
    for vertex_i in range(v_n):  # Iterate over all vertices
        lookup_start: int = NI[vertex_i]
        lookup_end: int = NI[vertex_i + 1]  
        arr = VF[lookup_start:lookup_end]
        
        vertex_to_face[vertex_i] = arr

    return vertex_to_face


## Vertex-to-Vertex Relations

In [5]:
def vertex_to_vertex(f) -> dict:
    adj_list: list =  igl.adjacency_list(f)
    adj_dict: dict = {i: arr for i, arr in enumerate(adj_list)}
    return adj_dict

## Bunny

In [6]:
def print_first_k_entries(data: dict, k: int):
    count = 0
    for key, value in data.items():
        print(f"{key} : {value}")
        count += 1
        if count >= k:
            break

In [7]:
v_to_f = vertex_to_face(bunny_f, len(bunny_v))
print("Vertex index to faces")
print_first_k_entries(v_to_f, 5)
print("---------------------------------")
print("Test. Check if vertex 0 is in above faces")
print(bunny_f[849], bunny_f[850], bunny_f[912], bunny_f[944], bunny_f[945])
print("\n---------------------------------\n")
v_to_v = vertex_to_vertex(bunny_f)


print("Vertex to Vertex")
print_first_k_entries(v_to_v, 5)
print("---------------------------------")


'''
# Print the whole list
print(v_to_f)
print("\n---------------------------------\n")
print(v_to_v)
print("\n---------------------------------\n")
print(bunny_f)
'''

Vertex index to faces
0 : [849 850 912 944 945]
1 : [ 738  788 1278 1833]
2 : [ 248 2795 2857 2972 3408]
3 : [911 912 945 960 974 991]
4 : [ 971 2977 2984 3012 3013]
---------------------------------
Test. Check if vertex 0 is in above faces
[  0 542 525] [  0  24 542] [  3   0 525] [308  24   0] [  3 308   0]

---------------------------------

Vertex to Vertex
0 : [3, 24, 308, 525, 542]
1 : [415, 549, 551, 596]
2 : [134, 287, 465, 497, 1308]
3 : [0, 246, 308, 510, 525, 543]
4 : [406, 1235, 1368, 1371, 1375]
---------------------------------


'\n# Print the whole list\nprint(v_to_f)\nprint("\n---------------------------------\n")\nprint(v_to_v)\nprint("\n---------------------------------\n")\nprint(bunny_f)\n'

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh

In [8]:
def explode_mesh(v, f):
    # apparently this is faster than appending to numpy 
    #https://stackoverflow.com/questions/29839350/numpy-append-vs-python-append
    exploded_v: list = []
    exploded_f: list = []
    available_i: int = 0
    index_mapping: dict = {}  # To store the mapping from original to exploded indices

    f_len = len(f)
    for i in range(f_len):
        cur_face = f[i]
        new_face = []
        
        for v_i in cur_face:
            exploded_v.append(v[v_i])
            new_face.append(available_i)
            
            # Create the mapping
            if v_i not in index_mapping:
                index_mapping[v_i] = []
            #Update the mapping
            index_mapping[v_i].append(available_i)

            available_i += 1
        
        exploded_f.append(new_face)

    return np.array(exploded_v), np.array(exploded_f), index_mapping


# scale to visualize normals
SPHERE_NORMAL_SCALE = 0.01
CUBE_NORMAL_SCALE = 0.5

### Flat Shading

In [12]:

def flat_shading(orig_v, orig_f, wireframe = True, normal_scale = 0.01):

    # Explode mesh then computer normals
    v, f, map = explode_mesh(orig_v, orig_f)
    n = igl.per_face_normals(v, f, np.empty((0, 3)))
    
    p = meshplot.plot(v, f, n=n, shading={"flat": True, "wireframe": wireframe})

    # so we have len(normals) = len(faces), but we have more vertices. 
    # so copy normal for each vertix in exploded faces.
    vertex_normals = np.zeros_like(v)
    for i, face in enumerate(f):
        for vertex in face:
            vertex_normals[vertex] += n[i]
        
    # Add lines for normals
    p.add_lines(v, v + normal_scale * vertex_normals)
    

flat_shading(sphere_v, sphere_f, False, SPHERE_NORMAL_SCALE)
flat_shading(cube_v, cube_f, True,CUBE_NORMAL_SCALE)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-vertex Shading

In [14]:
def per_vertex_shading(v, f, wireframe = True, normal_scale = 0.01):
    n = igl.per_vertex_normals(v, f)
    p = meshplot.plot(v, f, n=n, shading={"flat": False, "wireframe": wireframe})
    p.add_lines(v, v + normal_scale * n)


per_vertex_shading(cube_v, cube_f, normal_scale = CUBE_NORMAL_SCALE)
per_vertex_shading(sphere_v, sphere_f, normal_scale = SPHERE_NORMAL_SCALE)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Per-corner Shading

In [15]:
def per_corner_shading(v, f, threshold, wireframe=True, normal_scale=0.01):
    exploded_v, exploded_f, map = explode_mesh(v, f)

    corner_normals = igl.per_corner_normals(v, f, threshold)
    

    p = meshplot.plot(exploded_v, exploded_f, n=corner_normals, shading={"flat": False,"wireframe": wireframe})
    p.add_lines(exploded_v, exploded_v + normal_scale * corner_normals)
    


print("Cube with 20 degrees")
per_corner_shading(cube_v, cube_f, 20, False, CUBE_NORMAL_SCALE)
print("Cube with 100 degrees")
per_corner_shading(cube_v, cube_f, 100, False, CUBE_NORMAL_SCALE)
print("Sphere with 10 degrees")
per_corner_shading(sphere_v, sphere_f, 10, False, SPHERE_NORMAL_SCALE)
print("Sphere with 100 degrees")
per_corner_shading(sphere_v, sphere_f, 100, False, SPHERE_NORMAL_SCALE)

Cube with 20 degrees


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Cube with 100 degrees


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Sphere with 10 degrees


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Sphere with 100 degrees


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Connected Components

In [16]:
def connected_components(v, f):
    c=igl.facet_components(f)
    meshplot.plot(v,f,c=c)
    return c

def print_count_components(c):
    c = list(c)
    max_comp = max(c) + 1
    
    print(f"Number of component: {max_comp} (starts from 1)")
    for el in range(0, max_comp):
        print(f"Component {el+1} - {c.count(el)}")


c = connected_components(coffeecup_v, coffeecup_f)
print_count_components(c)
c = connected_components(car_v, car_f)
print_count_components(c)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.253649…

Number of component: 2 (starts from 1)
Component 1 - 3360
Component 2 - 2304


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.194198…

Number of component: 11 (starts from 1)
Component 1 - 90
Component 2 - 192
Component 3 - 192
Component 4 - 13216
Component 5 - 704
Component 6 - 1088
Component 7 - 1088
Component 8 - 1088
Component 9 - 1088
Component 10 - 736
Component 11 - 736


## A simple subdivision scheme

In [34]:
import math  

def sqrt_three_subdivision(orig_v, orig_f):
    # step 1
    middle_points = igl.barycenter(orig_v, orig_f)
    v = np.vstack((orig_v, middle_points)) # added new vertices where V = [orig_v , M_f]
    
    # create new faces and append
    f = []
    for face_i, face in enumerate(orig_f):
        v0_i, v1_i, v2_i = face

        middle_point_index = face_i + len(orig_v)
        f.append([v0_i, v1_i, middle_point_index]) 
        f.append([v0_i, v2_i, middle_point_index])  
        f.append([v1_i, v2_i, middle_point_index])  

    # step 2 calculating new poisituion of poinnts
    new_ps = np.zeros_like(orig_v)
    
    vertex_to_vertex_old = vertex_to_vertex(orig_f)
    for i, old_v in enumerate(orig_v):
        v_neighbours = vertex_to_vertex_old[i]
        n = len(v_neighbours)
        a_n = (4 - 2 * math.cos(2 * math.pi / n)) / 9

        # do the summation in the formula
        sum = np.empty(3)
        for j in range(n):
            sum += orig_v[v_neighbours[j]]
        # sum holds the summation of neighbours
        p = (1 - a_n) * old_v + (a_n / n) * sum
        new_ps[i] = p
        
    v = np.vstack((new_ps, middle_points))

    #step 3 flippping
    # i have f: list with only new faces. orig_f: np,array with origin faces
    # orig_v: np,array with orig vertices and v: new position vertices followed by middle points
    TT, TT_I = igl.triangle_triangle_adjacency(orig_f)
    flipped_faces = []

    for i, face in enumerate(orig_f):
        for edge_index in range(3):
            neighbor_triangle = TT[i][edge_index]
            if neighbor_triangle == -1:  # bboundary 
                continue

            # Shared edges
            edge_vertices = [face[edge_index], face[(edge_index + 1) % 3]]
            shared_triangle = orig_f[neighbor_triangle]
            # middle [points
            middle_index_1 = i + len(orig_v)  #  middle pointA
            middle_index_2 = neighbor_triangle + len(orig_v)  # middle point B
            
            # Flip the edge
            flipped_faces.append([edge_vertices[0], middle_index_1, middle_index_2])
            flipped_faces.append([edge_vertices[1], middle_index_1, middle_index_2])

    f = np.array(flipped_faces)
            
    
    return v, f


old_v, old_f = cube_v, cube_f
v, f = sqrt_three_subdivision(old_v, old_f )
print("CUBE with subdivison iteration 1")
p = meshplot.plot(v, f, shading={"wireframe": True})
p = meshplot.plot(old_v, old_f, shading={"wireframe": True})

print("SPhere with subdivison iteration 1")
old_v, old_f = sphere_v, sphere_f
v, f = sqrt_three_subdivision(old_v, old_f )
p = meshplot.plot(v, f, shading={"wireframe": True})
p = meshplot.plot(old_v, old_f, shading={"wireframe": True})
print("please check cell below this")
#  ---->

CUBE with subdivison iteration 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

SPhere with subdivison iteration 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.9802322…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

please check cell below this


In [32]:
# so because of step 2, my the old point get moved. This messes up the sphere a lot.
# Here is the version with commented out step 2 and looks much better

def sqrt_three_subdivision(orig_v, orig_f):
    # step 1
    middle_points = igl.barycenter(orig_v, orig_f)
    v = np.vstack((orig_v, middle_points)) # added new vertices where V = [orig_v , M_f]
    
    # create new faces and append
    f = []
    for face_i, face in enumerate(orig_f):
        v0_i, v1_i, v2_i = face

        middle_point_index = face_i + len(orig_v)
        f.append([v0_i, v1_i, middle_point_index]) 
        f.append([v0_i, v2_i, middle_point_index])  
        f.append([v1_i, v2_i, middle_point_index])  

    # step 2 calculating new poisituion of poinnts
    # new_ps = np.zeros_like(orig_v)
    
    # vertex_to_vertex_old = vertex_to_vertex(orig_f)
    # for i, old_v in enumerate(orig_v):
    #     v_neighbours = vertex_to_vertex_old[i]
    #     n = len(v_neighbours)
    #     a_n = (4 - 2 * math.cos(2 * math.pi / n)) / 9

    #     # do the summation in the formula
    #     sum = np.empty(3)
    #     for j in range(n):
    #         sum += orig_v[v_neighbours[j]]
    #     # sum holds the summation of neighbours
    #     p = (1 - a_n) * old_v + (a_n / n) * sum
    #     new_ps[i] = p
        
    # v = np.vstack((new_ps, middle_points))

    #step 3 flippping
    # i have f: list with only new faces. orig_f: np,array with origin faces
    # orig_v: np,array with orig vertices and v: new position vertices followed by middle points
    TT, TT_I = igl.triangle_triangle_adjacency(orig_f)
    flipped_faces = []

    for i, face in enumerate(orig_f):
        for edge_index in range(3):
            neighbor_triangle = TT[i][edge_index]
            if neighbor_triangle == -1:  # bboundary 
                continue

            # Shared edges
            edge_vertices = [face[edge_index], face[(edge_index + 1) % 3]]
            shared_triangle = orig_f[neighbor_triangle]

            # middle [points
            middle_index_1 = i + len(orig_v)  #  middle pointA
            middle_index_2 = neighbor_triangle + len(orig_v)  # middle point B

            # Flip the edge
            flipped_faces.append([edge_vertices[0], middle_index_1, middle_index_2])
            flipped_faces.append([edge_vertices[1], middle_index_1, middle_index_2])

    f = np.array(flipped_faces)
            
    
    return v, f


old_v, old_f = cube_v, cube_f
v, f = sqrt_three_subdivision(old_v, old_f )
print("CUBE with subdivison iteration 1")
p = meshplot.plot(v, f, shading={"wireframe": True})
p = meshplot.plot(old_v, old_f, shading={"wireframe": True})

print("SPhere with subdivison iteration 1")
old_v, old_f = sphere_v, sphere_f
v, f = sqrt_three_subdivision(old_v, old_f )
p = meshplot.plot(v, f, shading={"wireframe": True})
p = meshplot.plot(old_v, old_f, shading={"wireframe": True})

CUBE with subdivison iteration 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

SPhere with subdivison iteration 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…